# Парсинг текстов с сайтов и соцсетей

Этот блокнот загружает скрипты из репозитория и помогает быстро получить тексты из:
- статей Фонтанки
- отзывов Яндекса
- групп ВК

Ниже — установка зависимостей, импорт функций и интерактивные формы для ввода URL/токенов.


In [ ]:
#@title Установка зависимостей
!pip -q install requests beautifulsoup4 lxml ipywidgets

import os
import sys
from pathlib import Path

REPO_URL = "https://github.com/OWNER/digital_identity"
REPO_DIR = Path("/content/digital_identity")

if not REPO_DIR.exists():
    !git clone --depth 1 {REPO_URL} {REPO_DIR}

sys.path.insert(0, str(REPO_DIR))


In [ ]:
#@title Импорт парсеров
from parsers import fetch_fontanka_article, fetch_yandex_reviews, VKGroupParser


In [ ]:
#@title Интерактивный запуск
import ipywidgets as widgets
from IPython.display import display

parser_dropdown = widgets.Dropdown(
    options=[
        ("Фонтанка", "fontanka"),
        ("Яндекс отзывы", "yandex"),
        ("ВК группа", "vk"),
    ],
    description="Парсер:"
)
url_input = widgets.Text(
    value="",
    description="URL/домен:",
    placeholder="https://...",
    layout=widgets.Layout(width="80%"),
)
token_input = widgets.Password(
    value="",
    description="VK токен:",
    placeholder="Требуется только для ВК",
)
max_items = widgets.IntSlider(
    value=20,
    min=1,
    max=200,
    step=1,
    description="Лимит:"
)
run_button = widgets.Button(description="Запустить")
output = widgets.Output()

def run_parser(_):
    output.clear_output()
    with output:
        parser = parser_dropdown.value
        if parser == "fontanka":
            article = fetch_fontanka_article(url_input.value)
            print(article.title)
            print(article.published_at)
            print(article.text[:3000])
        elif parser == "yandex":
            reviews = fetch_yandex_reviews(url_input.value, max_reviews=max_items.value)
            for review in reviews:
                print(review.text)
                print("-" * 20)
        else:
            if not token_input.value:
                raise ValueError("Нужен VK токен.")
            vk = VKGroupParser(token_input.value)
            for post in vk.iter_posts(url_input.value, total=max_items.value):
                print(post.text)
                print("-" * 20)

run_button.on_click(run_parser)
display(parser_dropdown, url_input, token_input, max_items, run_button, output)
